In [1]:
## Colab
# from google.colab import drive
# drive.mount('/gdrive')
# gData = '/gdrive/MyDrive/Data'

## Kaggle
# gData = '/kaggle/input'

## Mac mini (M1 2020)
# gData = '/Users/achilles/Workspace/Data'

## HP-Z4-G4 Ubuntu
gData = "/mnt/Storage/Xuchu_Liu/Workspace/Data"

## preprocess_dsb2018.py

In [2]:
import os
from glob import glob

import cv2
import numpy as np
from tqdm import tqdm

In [3]:
# img_size = 96

# data_path = os.path.join(gData, 'data-science-bowl-2018')


# paths = glob(os.path.join(data_path, 'stage1_train', '*'))

# os.makedirs(os.path.join(data_path, 'dsb2018_%d/images' % img_size), exist_ok=True)
# os.makedirs(os.path.join(data_path, 'dsb2018_%d/masks/0' % img_size), exist_ok=True)

# for i in tqdm(range(len(paths))):
#     path = paths[i]
#     img = cv2.imread(os.path.join(path, 'images',
#                      os.path.basename(path) + '.png'))
#     mask = np.zeros((img.shape[0], img.shape[1]))
#     for mask_path in glob(os.path.join(path, 'masks', '*')):
#         mask_ = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) > 127
#         mask[mask_] = 1
#     if len(img.shape) == 2:
#         img = np.tile(img[..., None], (1, 1, 3))
#     if img.shape[2] == 4:
#         img = img[..., :3]
#     img = cv2.resize(img, (img_size, img_size))
#     mask = cv2.resize(mask, (img_size, img_size))
#     cv2.imwrite(os.path.join(data_path, 'dsb2018_%d/images' % img_size,
#                 os.path.basename(path) + '.png'), img)
#     cv2.imwrite(os.path.join(data_path, 'dsb2018_%d/masks/0' % img_size,
#                 os.path.basename(path) + '.png'), (mask * 255).astype('uint8'))

## train.py

In [4]:
import argparse
from collections import OrderedDict

import pandas as pd
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import yaml

from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler

## archs.py

In [5]:
class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out

In [6]:
# vggblock = VGGBlock(1, 32, 32)
# vggblock.eval()

In [7]:
class UNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])
        self.conv2_2 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv1_3 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv0_4 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])

        self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x2_0 = self.conv2_0(self.pool(x1_0))
        x3_0 = self.conv3_0(self.pool(x2_0))
        x4_0 = self.conv4_0(self.pool(x3_0))

        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, self.up(x1_3)], 1))

        output = self.final(x0_4)
        return output

In [8]:
# unet = UNet(2,1)
# unet.eval()

In [9]:
class NestedUNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, deep_supervision=False, **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]

        self.deep_supervision = deep_supervision

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])

        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4]

        else:
            output = self.final(x0_4)
            return output

In [10]:
# nestedunet = NestedUNet(2,1)
# nestedunet.eval()

## losses.py

In [11]:
class BCEDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        bce = F.binary_cross_entropy_with_logits(input, target)
        smooth = 1e-5
        input = torch.sigmoid(input)
        num = target.size(0)
        input = input.view(num, -1)
        target = target.view(num, -1)
        intersection = (input * target)
        dice = (2. * intersection.sum(1) + smooth) / (input.sum(1) + target.sum(1) + smooth)
        dice = 1 - dice.sum() / num
        return 0.5 * bce + dice

In [12]:
# from LovaszSoftmax.pytorch.lovasz_losses import lovasz_hinge
# class LovaszHingeLoss(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, input, target):
#         input = input.squeeze(1)
#         target = target.squeeze(1)
#         loss = lovasz_hinge(input, target, per_image=True)

#         return loss

## dataset.py

In [13]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_ids, img_dir, mask_dir, img_ext, mask_ext, num_classes, transform=None):
        """
        Args:
            img_ids (list): Image ids.
            img_dir: Image file directory.
            mask_dir: Mask file directory.
            img_ext (str): Image file extension.
            mask_ext (str): Mask file extension.
            num_classes (int): Number of classes.
            transform (Compose, optional): Compose transforms of albumentations. Defaults to None.
        
        Note:
            Make sure to put the files as the following structure:
            <dataset name>
            ├── images
            |   ├── 0a7e06.jpg
            │   ├── 0aab0a.jpg
            │   ├── 0b1761.jpg
            │   ├── ...
            |
            └── masks
                ├── 0
                |   ├── 0a7e06.png
                |   ├── 0aab0a.png
                |   ├── 0b1761.png
                |   ├── ...
                |
                ├── 1
                |   ├── 0a7e06.png
                |   ├── 0aab0a.png
                |   ├── 0b1761.png
                |   ├── ...
                ...
        """
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_ext = img_ext
        self.mask_ext = mask_ext
        self.num_classes = num_classes
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        
        img = cv2.imread(os.path.join(self.img_dir, img_id + self.img_ext))

        mask = []
        for i in range(self.num_classes):
            mask.append(cv2.imread(os.path.join(self.mask_dir, str(i),
                        img_id + self.mask_ext), cv2.IMREAD_GRAYSCALE)[..., None])
        mask = np.dstack(mask)

        if self.transform is not None:
            augmented = self.transform(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        
        img = img.astype('float32') / 255
        img = img.transpose(2, 0, 1)
        mask = mask.astype('float32') / 255
        mask = mask.transpose(2, 0, 1)
        
        return img, mask, {'img_id': img_id}

## metrics.py

In [14]:
def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)

In [15]:
def dice_coef(output, target):
    smooth = 1e-5

    output = torch.sigmoid(output).view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)

## utils

In [16]:
def str2bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [17]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## python train.py --dataset dsb2018_96 --arch NestedUNet

### config = vars(parse_args())

In [19]:
name = 'dsb2018_96_NestedUNet_woDS'
epochs = 100
batch_size = 16
# arch = 'NestedUNet'
arch = 'UNet'
deep_supervision = False
input_channels = 3
num_classes = 1
input_w = 96
input_h =96
loss = 'BCEDiceLoss'
dataset = 'dsb2018_96'
img_ext = '.png'
mask_ext = '.png'
optimizer = 'SGD'
lr = 0.001
momentum = 0.9
weight_decay = 0.0001
nesterov = False
scheduler = 'CosineAnnealingLR'
min_lr = 1e-05
factor = 0.1
patience = 2
milestones = '1,2'
gamma = 0.6666666666666666
early_stopping = -1
num_workers = 4

### define loss function (criterion)

In [20]:
if loss == 'BCEWithLogitsLoss':
    criterion = nn.BCEWithLogitsLoss().cuda()
else:
    criterion = BCEDiceLoss().cuda()

cudnn.benchmark = True

In [21]:
print("=> creating model %s" % arch)
model = NestedUNet(num_classes, input_channels, deep_supervision)

model = model.cuda()

=> creating model UNet


In [22]:
params = filter(lambda p: p.requires_grad, model.parameters())

if optimizer == 'Adam':
    optimizer = optim.Adam(
        params, lr=lr, weight_decay=weight_decay)
elif optimizer == 'SGD':
    optimizer = optim.SGD(params, lr=lr, momentum=momentum,
                          nesterov=nesterov, weight_decay=weight_decay)

In [23]:
if scheduler == 'CosineAnnealingLR':
    scheduler = lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs, eta_min=min_lr)
elif scheduler == 'ReduceLROnPlateau':
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=patience,
                                               verbose=1, min_lr=min_lr)
elif scheduler == 'MultiStepLR':
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[int(e) for e in milestones.split(',')], gamma=gamma)
elif scheduler == 'ConstantLR':
    scheduler = None

### Data loading code

In [24]:
data_path = os.path.join(gData, 'data-science-bowl-2018')

img_ids = glob(os.path.join(data_path, dataset, 'images', '*' + img_ext))
img_ids = [os.path.splitext(os.path.basename(p))[0] for p in img_ids]

train_img_ids, val_img_ids = train_test_split(img_ids, test_size=0.2, random_state=41)

In [25]:
import albumentations as albu
train_transform = Compose([
    albu.RandomRotate90(),
    transforms.Flip(),
    OneOf([
        transforms.HueSaturationValue(),
        transforms.RandomBrightnessContrast(),
        # transforms.RandomContrast(),
    ], p=1),
    albu.Resize(input_h, input_w),
    transforms.Normalize(),
])

val_transform = Compose([
    albu.Resize(input_h, input_w),
    transforms.Normalize(),
])

In [26]:
train_dataset = Dataset(
    img_ids=train_img_ids,
    img_dir=os.path.join(data_path, dataset, 'images'),
    mask_dir=os.path.join(data_path, dataset, 'masks'),
    img_ext=img_ext,
    mask_ext=mask_ext,
    num_classes=num_classes,
    transform=train_transform)
val_dataset = Dataset(
    img_ids=val_img_ids,
    img_dir=os.path.join(data_path, dataset, 'images'),
    mask_dir=os.path.join(data_path, dataset, 'masks'),
    img_ext=img_ext,
    mask_ext=mask_ext,
    num_classes=num_classes,
    transform=val_transform)

In [27]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False)

In [28]:
log = OrderedDict([
    ('epoch', []),
    ('lr', []),
    ('loss', []),
    ('iou', []),
    ('val_loss', []),
    ('val_iou', []),
])

In [29]:
def train(train_loader, model, criterion, optimizer):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    model.train()

    pbar = tqdm(total=len(train_loader))
    for input, target, _ in train_loader:
        input = input.cuda()
        target = target.cuda()

        # compute output
        if deep_supervision:
            outputs = model(input)
            loss = 0
            for output in outputs:
                loss += criterion(output, target)
            loss /= len(outputs)
            iou = iou_score(outputs[-1], target)
        else:
            output = model(input)
            loss = criterion(output, target)
            iou = iou_score(output, target)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])

In [30]:
def validate(val_loader, model, criterion):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        pbar = tqdm(total=len(val_loader))
        for input, target, _ in val_loader:
            input = input.cuda()
            target = target.cuda()

            # compute output
            if deep_supervision:
                outputs = model(input)
                loss = 0
                for output in outputs:
                    loss += criterion(output, target)
                loss /= len(outputs)
                iou = iou_score(outputs[-1], target)
            else:
                output = model(input)
                loss = criterion(output, target)
                iou = iou_score(output, target)

            avg_meters['loss'].update(loss.item(), input.size(0))
            avg_meters['iou'].update(iou, input.size(0))

            postfix = OrderedDict([
                ('loss', avg_meters['loss'].avg),
                ('iou', avg_meters['iou'].avg),
            ])
            pbar.set_postfix(postfix)
            pbar.update(1)
        pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])

In [31]:
best_iou = 0
trigger = 0
for epoch in range(epochs):
    print('Epoch [%d/%d]' % (epoch, epochs))

    # train for one epoch
    train_log = train(train_loader, model, criterion, optimizer)
    # evaluate on validation set
    val_log = validate(val_loader, model, criterion)

    if scheduler == 'CosineAnnealingLR':
        scheduler.step()
    elif scheduler == 'ReduceLROnPlateau':
        scheduler.step(val_log['loss'])

    print('loss %.4f - iou %.4f - val_loss %.4f - val_iou %.4f'
          % (train_log['loss'], train_log['iou'], val_log['loss'], val_log['iou']))

    log['epoch'].append(epoch)
    log['lr'].append(lr)
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])

    pd.DataFrame(log).to_csv('models/%s/log.csv' %
                             name, index=False)

    trigger += 1

    if val_log['iou'] > best_iou:
        torch.save(model.state_dict(), 'models/%s/model.pth' %
                   name)
        best_iou = val_log['iou']
        print("=> saved best model")
        trigger = 0

    # early stopping
    if early_stopping >= 0 and trigger >= early_stopping:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()

Epoch [0/100]


100%|██████████████████████| 9/9 [00:00<00:00, 12.43it/s, loss=1.13, iou=0.0102]


loss 1.1006 - iou 0.2262 - val_loss 1.1272 - val_iou 0.0102
=> saved best model
Epoch [1/100]


100%|█████████████████████████| 9/9 [00:00<00:00, 21.04it/s, loss=1.08, iou=0.1]


loss 0.9674 - iou 0.3797 - val_loss 1.0838 - val_iou 0.1004
=> saved best model
Epoch [2/100]


...

loss 0.2009 - iou 0.7785 - val_loss 0.1685 - val_iou 0.8242
Epoch [98/100]


100%|██████████████████████| 9/9 [00:00<00:00, 18.78it/s, loss=0.156, iou=0.831]


loss 0.2031 - iou 0.7838 - val_loss 0.1564 - val_iou 0.8311
Epoch [99/100]


100%|██████████████████████| 9/9 [00:00<00:00, 18.94it/s, loss=0.152, iou=0.836]

loss 0.2140 - iou 0.7737 - val_loss 0.1523 - val_iou 0.8365


## val.py

In [32]:
model.load_state_dict(torch.load('models/%s/model.pth' % name))
model.eval()

NestedUNet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (up): Upsample(scale_factor=2.0, mode=bilinear)
  (conv0_0): VGGBlock(
    (relu): ReLU(inplace=True)
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv1_0): VGGBlock(
    (relu): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2_0): VGGBlock(
    (relu): ReLU(inplace=True)
    

In [33]:
val_transform = Compose([
    albu.Resize(input_h, input_w),
    transforms.Normalize(),
])

In [34]:
val_dataset = Dataset(
    img_ids=val_img_ids,
    img_dir=os.path.join(data_path, dataset, 'images'),
    mask_dir=os.path.join(data_path, dataset, 'masks'),
    img_ext=img_ext,
    mask_ext=mask_ext,
    num_classes=num_classes,
    transform=val_transform)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False)

In [35]:
avg_meter = AverageMeter()

for c in range(num_classes):
    os.makedirs(os.path.join('outputs', name, str(c)), exist_ok=True)
with torch.no_grad():
    for input, target, meta in tqdm(val_loader, total=len(val_loader)):
        input = input.cuda()
        target = target.cuda()

        # compute output
        if deep_supervision:
            output = model(input)[-1]
        else:
            output = model(input)

        iou = iou_score(output, target)
        avg_meter.update(iou, input.size(0))

        output = torch.sigmoid(output).cpu().numpy()

        for i in range(len(output)):
            for c in range(num_classes):
                cv2.imwrite(os.path.join('outputs', name, str(c), meta['img_id'][i] + '.jpg'),
                            (output[i, c] * 255).astype('uint8'))

print('IoU: %.4f' % avg_meter.avg)

torch.cuda.empty_cache()


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 16.21it/s]

IoU: 0.8372
